In [4]:
from csv import reader
from math import sqrt
from json import dump
import operator

### считываем содержимое файла и помещаем его в список
def csv_reader(path):
    lst = []
    with open(path, "r") as f_obj:
        rdr = reader(f_obj)
        next(rdr)
        for row in rdr:
            row.pop(0)
            lst.append(row)
    return lst

### Используем метрку косинуса в качестве метрики сходства
def sim(u, v):
    numerator = 0
    d1 = 0
    d2 = 0
    for i in range(len(u)):
        if u[i] == -1 or v[i] == -1:
            continue
        numerator += u[i] * v[i]
        d1 += u[i] ** 2
        d2 += v[i] ** 2
    return round(numerator / (sqrt(d1) * sqrt(d2)), 3)

### Подсчет средней оценки пользователя
def avg(u):
    avg_u = 0
    cnt_u = 0
    for i in range(len(u)):
        if u[i] != -1:
            avg_u += u[i]
            cnt_u += 1
    return round(avg_u / cnt_u, 3)

### Сортируем результат по убыванию значения
def kNN(sim, numb):
    sim = sorted(sim, key=operator.itemgetter(1), reverse=True)
    sim = sim[:numb]
    return sim

### Для пользователя u рассчитывается оценка фильма i
def r_u_i(u, i, data, sims_knn):
    n1 = 0
    denominator = 0
    for v in sims_knn:
        if data[v[0]][i] == -1:
            continue
        n1 += v[1]*(data[v[0]][i] - avg(data[v[0]]))
        denominator += abs(v[1])
    return round(avg(u) + n1 / denominator, 3)

### Записываем все фильмы, которые 4 самых похожих пользователя с искомым пользователем, посмотрели у себя дома в выходной день и котому они поставили оценку выше, чем их средняя оценка за все фильмы, и помещаем все фильмы, удовлетворяющие этим условиям, в dictionary (Ключ - это индекс фильма, а значение - это оценка, которую бы поставил этому фильму искомый пользователь). Сортируем dictionary по убыванию предполагаемой оценки и возвращаем tuple
def recommend(user, data, sims_knn, place, day, new_rnk):
    rec = {}
    for v in sims_knn:
        for i in range(len(data[0])):
            if data[user][i] == -1 and place[v[0]][i] == ' h' and (day[v[0]][i] == ' Sat' or day[v[0]][i] == ' Sun') \
                        and data[v[0]][i] > avg(data[v[0]]):
                index = "Movie " + str(i + 1)
                rec[index] = new_rnk[index]
    rec = sorted(rec.items(), key=operator.itemgetter(1), reverse=True)
    return rec

### Создаем json-файл с выводом предполагаемых оценок и в качестве рекомендации предлагаем первый фильм (с наивысшей предполагаемой оценкой) из tuple для этого пользователя
def make_json(user, new_rnk, recom):
    json_lst = {
        "user": "User" + str(user + 1),
        "1": new_rnk,
        "2": {
            recom[0][0]: recom[0][1]
        }
    }
    return json_lst

### Вычисление отсутствующих оценок и генерация рекомендации
def rate_film(user, data, place, day):
    user -= 1
    sims = []
    for j in range(len(data)):
        if user != j:
            sims.append((j, sim(data[user], data[j])))
    rnk_all = {}
    sims_knn = kNN(sims, 4)
    for i in range(len(data[0])):
        if data[user][i] == -1:
            rank = r_u_i(data[user], i, data_list, sims_knn)
            rnk_all["Movie " + str(i + 1)] = rank
    recom = recommend(user, data, sims_knn, place_list, day_list, rnk_all)
    json_file = make_json(user, rnk_all, recom)
    print(json_file)
    with open('user.json', 'w') as outfile:
        dump(json_file, outfile)


data_list = csv_reader("data.csv")
day_list = csv_reader("context_day.csv")
place_list = csv_reader("context_place.csv")

for i in range(len(data_list)):
    for j in range(len(data_list[0])):
        data_list[i][j] = int(data_list[i][j])

user_id = int(input("Please enter a user id: "))
rate_film(user_id, data_list, place_list, day_list)


Please enter a user id: 2
{'user': 'User2', '1': {'Movie 7': 3.095, 'Movie 12': 3.348, 'Movie 17': 3.095, 'Movie 21': 2.801, 'Movie 23': 1.787, 'Movie 27': 2.344}, '2': {'Movie 12': 3.348}}
